<a href="https://colab.research.google.com/github/Chachoummm/chess_engine/blob/main/Train_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#libraries
import glob
import os
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.utils import shuffle


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:

# comme c'est trop lourd c'est pas sur github donc le modèle a été entrainé en local et puis on y a fait appel sur colab à partir de Github
train = pd.read_csv('/content/gdrive/MyDrive/final_dataset.csv', index_col=None, header=0)
#train = pd.concat(li, axis=0, ignore_index=True)



In [ ]:
train = shuffle(train)

In [ ]:
print (train.shape)

(610240, 193)


In [ ]:
print (train.head())

          a1    b1    c1    d1    e1  ... to_e8 to_f8 to_g8 to_h8 good_move
166370  None     q  None  None  None  ...   0.0   0.0   0.0   0.0     False
82799      R  None  None  None  None  ...   0.0   0.0   0.0   0.0      True
440393  None     R     B     R  None  ...   0.0   0.0   0.0   0.0     False
349346     R  None     B  None  None  ...   0.0   0.0   1.0   0.0     False
473712     R     N     B     Q     K  ...   0.0   0.0   0.0   0.0     False

[5 rows x 193 columns]


In [ ]:
#features transformations 
features = list(train.iloc[:, 0:192].columns)
X = train[features]
y = train['good_move']
categorical_columns = list(X.iloc[:, 0:63].columns)
numerical_columns = list(X.iloc[:, 64:192].columns)
feature_columns = []

for feature_name in categorical_columns:
  vocabulary = X[feature_name].unique()
  feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))


for feature_name in numerical_columns:
  feature_columns.append(tf.feature_column.numeric_column(feature_name,dtype = tf.float32))


In [ ]:
#input function :   
  
def make_input_fn(data_df, label_df, num_epochs = 10, shuffle = True, batch_size = 32):
  def input_function():
    ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))
    if shuffle:
      ds = ds.shuffle(1000)
    ds = ds.batch(batch_size).repeat(num_epochs)
    return ds
  return input_function 

In [ ]:
#split data into batches :  
  
def split_into_batches(df, batch_size=100000):
  nb_rows = len(df.index)
  intervals = []
  
  for i in range(0, nb_rows + 1, batch_size):
    intervals.append(i)
  
  if(intervals[-1] != nb_rows):
    intervals.append(nb_rows)
  
  batches_X = []
  batches_y = []
  
  for i in range(0, len(intervals) - 1):
    batches_X.append(train.iloc[intervals[i]:intervals[i + 1], :][features])
    batches_y.append(train.iloc[intervals[i]:intervals[i + 1], :]['good_move'])

  return batches_X, batches_y

batches_X, batches_y = split_into_batches(train)

In [ ]:
#model : 
linear_est = tf.estimator.LinearClassifier(feature_columns = feature_columns, model_dir='/content/gdrive/MyDrive/Saved_train_model',
    warm_start_from=None, loss_reduction=tf.losses.Reduction.SUM_OVER_BATCH_SIZE,
    sparse_combiner='sum')

In [ ]:
#Create a list with all the input function : 
input_functions = []
for df_X, df_y in zip(batches_X, batches_y):
  input_functions.append(make_input_fn(df_X, df_y))

print(len(input_functions))

7


In [ ]:
# train the model on all the input functions
i = 1
for input_function in input_functions:
  print('<======================================== NEW BATCH ========================================>')
  print('Batch: ' + str(i))
  i = i + 1
  linear_est.train(input_function)
  

In [ ]:
#Evaluate the model 

predict = 
# Evaluate the model on all the input functions
i = 1
for input_function in input_functions:
  print('<======================================== NEW BATCH ========================================>')
  print('Batch: ' + str(i))
  i = i + 1
  linear_est.train(input_function)

In [ ]:
# save the model
serving_input_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(
  tf.feature_column.make_parse_example_spec(feature_columns))

estimator_base_path = '/content/gdrive/MyDrive/Saved_train_model'
estimator_path = linear_est.export_saved_model(estimator_base_path, serving_input_fn)